Channel PCA captures most significant patterns across frames for each channel, potentially highlighting the most prominent changes or features in the video content for that specific channel.
**Do we want that or are there better ways to capture spatial features per frame?**

In [1]:
import os
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

In [2]:
def find_repo_root(path='.'):
    path = os.path.abspath(path)
    while not os.path.isdir(os.path.join(path, '.git')):
        parent = os.path.dirname(path)
        if parent == path:
            # We've reached the root of the file system without finding '.git'
            return None
        path = parent
    return path

repo_root = find_repo_root()
print("Repository Root:", repo_root)

Repository Root: C:\Users\marce\PycharmProjects\Brainvision_Project


In [3]:
def get_full_path(relative_path, repo_root):
    if not repo_root:
        raise ValueError("Repository root not found. Ensure you're inside a Git repository.")

    return os.path.join(repo_root, relative_path)


In [4]:
# step 1 function
def load_and_combine_tensors(stage_name, input_folder, num_videos):
    combined_tensor = []
    video_indices = {}

    for video_id in range(1, num_videos + 1):
        filename = f"{str(video_id).zfill(4)}_{stage_name}.pkl"
        file_path = os.path.join(input_folder, stage_name, filename)

        if os.path.exists(file_path):
            #print(f"Loading tensor from: {file_path}")
            with open(file_path, 'rb') as file:
                tensor = pickle.load(file)
                combined_tensor.append(tensor)
                # Track start and end indices for each video
                end_index = sum(t.shape[0] for t in combined_tensor)
                video_indices[str(video_id).zfill(4)] = (end_index - tensor.shape[0], end_index)

    if not combined_tensor:
        print("No tensors found to combine.")
        return None, None

    combined_tensor = np.concatenate(combined_tensor, axis=0)
    return combined_tensor, video_indices


In [5]:
# Step 2: globalized standardization (only based on training set)
def standardize_tensors(combined_tensor, video_indices, training_end_id='0005'):
    reshaped_tensor = combined_tensor.reshape(combined_tensor.shape[0], -1)
    scaler = StandardScaler()

    # Find the end index of the training set
    training_end_index = video_indices[training_end_id][1]

    # Fit the scaler only on the training set
    scaler.fit(reshaped_tensor[:training_end_index])

    # Transform both training and test sets
    standardized_data = scaler.transform(reshaped_tensor)
    
    return standardized_data.reshape(combined_tensor.shape)

In [6]:
# Step 3: Separate the standardized tensor back into individual tensors
def separate_standardized_tensor(standardized_tensor, video_indices):
    separated_tensors = {}
    for video_id, (start, end) in video_indices.items():
        separated_tensors[video_id] = standardized_tensor[start:end, :]
    return separated_tensors

In [7]:
# to get same number of PCs for each video: fit PCA on all videos with a given variance threshold. Find max number of components. Fit and transform PCA with max number of components. -> makes sure that variance captured in each video is >= variance_ratio.

def apply_fmpca_and_save(separated_tensors, stage_name, output_folder, variance_ratio):
    # Assuming 'separated_tensors' is your dictionary with video IDs as keys and feature maps as values
    print(f"Variance Ratio: {variance_ratio}")
    
    # Initialize a dictionary to store the final PCA results for each video
    final_pca_results = {}
    # Determine the spatial dimensions product from the first tensor
    first_feature_map = next(iter(separated_tensors.values()))
    spatial_dims_product = np.prod(first_feature_map.shape[2:4])  # Assuming spatial dimensions are in 3rd and 4th place
    
    # Step 1: Determine the maximum number of components needed, but only for training set
    max_components = 0
    training_video_ids = [vid for vid in separated_tensors if vid <= "0800"]
    for video_id in tqdm(training_video_ids, desc="Finding max. number of PCs..."):
        feature_maps = separated_tensors[video_id]
        for channel in range(feature_maps.shape[-1]):
            pca = PCA(n_components=variance_ratio)
            data_for_channel = feature_maps[..., channel].reshape(-1, spatial_dims_product)
            pca.fit(data_for_channel)
            max_components = max(max_components, pca.n_components_)
    
    print(f"Max. number of PCs: {max_components}")
    
    # Step 2: Apply PCA with the determined number of components
    for video_id, feature_maps in tqdm(separated_tensors.items(), desc="Performing PCA..."):
        pca_results = []
    
        # Loop over each channel
        for channel in range(feature_maps.shape[-1]):
            # Reshape the data for this channel
            data_for_pca = feature_maps[..., channel].reshape(-1, spatial_dims_product)
    
            # Apply PCA with the maximum number of components
            pca = PCA(n_components=max_components)
            pca_result = pca.fit_transform(data_for_pca)
    
            pca_results.append(pca_result)
    
        # Concatenate the PCA results from all channels for this video
        final_result = np.concatenate(pca_results, axis=1)
    
        # Store the result in the dictionary with the video ID as the key
        final_pca_results[video_id] = final_result
        # final_pca_results now contains the PCA-transformed data for each video
        # print(f"Processed Video ID: {video_id}, Resulting Shape: {final_result.shape}")
    print(variance_ratio)
    pca_folder = os.path.join(output_folder, f"PCA_channel_{variance_ratio}", stage_name)
    if not os.path.exists(pca_folder):
        os.makedirs(pca_folder)
        
    file_path = os.path.join(pca_folder, 'pca_results.pkl')
    with open(file_path, 'wb') as f:
        pickle.dump(final_pca_results, f)
    
    print(f"{stage_name} PCs stored in: {file_path}")

In [8]:
def process_stage_for_pca(input_folder, output_folder, stage_name):
    """
    Process all videos of a given stage: standardize, apply PCA, and save the PCA-transformed tensors.
    Args:
    - input_folder: Folder containing the pre-processed videos.
    - output_folder: Folder to save PCA results.
    - stage_name: Name of the stage to process.
    Returns:
    - DataFrame containing metadata (video ID and variance captured).
    """
     # Use the current working directory or a known absolute path
    current_working_directory = os.getcwd()
    stage_folder = os.path.join(current_working_directory, input_folder, stage_name)
    print("Attempting to access:", stage_folder)

    if not os.path.exists(stage_folder):
        print("Directory not found:", stage_folder)
        return None
    # Calculate the number of video files in the folder
    num_videos = len([f for f in os.listdir(stage_folder) if os.path.isfile(os.path.join(stage_folder, f))])
    print(f"Number of videos found: {num_videos}")

    # Step 1: Load and combine tensors
    combined_tensor, video_indices = load_and_combine_tensors(stage_name, input_folder, num_videos)
    print("Step 1 done.")
    # Step 2: Globally standardize the tensor
    standardized_tensor = standardize_tensors(combined_tensor, video_indices)
    print("Step 2 done.")
    # Step 3: Separate the standardized tensor back into individual tensors
    separated_tensors = separate_standardized_tensor(standardized_tensor, video_indices)
    print("Step 3 done.")
    # Step 4: Apply PCA to each tensor and save the result
    apply_fmpca_and_save(separated_tensors, stage_name, output_folder, variance_ratio)
    print("Step 4 done.")

In [9]:
print(os.getcwd())
print(repo_root)

C:\Users\marce\PycharmProjects\Brainvision_Project
C:\Users\marce\PycharmProjects\Brainvision_Project


In [10]:
# Example usage
input_folder = 'preprocessed_videos_30frames'
output_folder = os.getcwd()
stages = ["stage_1", "stage_2", "stage_3", "stage_4", "stage_5"]
variance_ratio = 0.95

# Iterate over each stage and process it
# for stage in stages:
#     print(f"Processing {stage}...")
#     process_stage_for_pca(input_folder, output_folder, stage)


In [11]:
combined_tensor, video_indices = load_and_combine_tensors("stage_5", input_folder, 5)
standardized_tensor = standardize_tensors(combined_tensor, video_indices)

In [12]:
print(standardized_tensor.shape)

(150, 1, 7, 7, 2048)


In [13]:
# Function to slice feature maps into four equal parts
def slice_feature_maps(feature_maps):
    _, _, height, width, _ = feature_maps.shape

    # Calculate midpoints
    mid_height = height // 2
    mid_width = width // 2

    # Slicing the feature maps into four equal parts
    top_left = feature_maps[:, :, :mid_height, :mid_width, :]
    top_right = feature_maps[:, :, :mid_height, mid_width:, :]
    bottom_left = feature_maps[:, :, mid_height:, :mid_width, :]
    bottom_right = feature_maps[:, :, mid_height:, mid_width:, :]

    return top_left, top_right, bottom_left, bottom_right

In [14]:
# Apply the function to your feature maps
top_left, top_right, bottom_left, bottom_right = slice_feature_maps(standardized_tensor)

In [15]:
print(top_left.shape)
print(top_right.shape)
print(bottom_left.shape)
print(bottom_right.shape)


(150, 1, 3, 3, 2048)
(150, 1, 3, 4, 2048)
(150, 1, 4, 3, 2048)
(150, 1, 4, 4, 2048)


In [16]:
def apply_pca_and_concatenate(slices, n_components=0.95):
    pca_results = []
    
    for feature_maps in slices:
        # Reshape the slice to 2D array
        # Reshape and concatenate the feature maps for each 30-frame segment
        n_videos = feature_maps.shape[0] // 30
        reshaped_data = np.zeros((n_videos, feature_maps.shape[2] * feature_maps.shape[3] * feature_maps.shape[4] * 30))
        
        for i in range(n_videos):
            # Flatten and concatenate the feature maps for each segment
            video = feature_maps[i*30:(i+1)*30].reshape(-1)
            reshaped_data[i, :] = video

        
        # Apply PCA
        pca = PCA(n_components=n_components)
        pca_result = pca.fit_transform(reshaped_data)
        
        # Append the PCA result
        pca_results.append(pca_result)

    # Concatenate the PCA results from all slices
    final_result = np.concatenate(pca_results, axis=1)
    
    return final_result

In [17]:
slices = [top_left, top_right, bottom_left, bottom_right]
final_pca_result = apply_pca_and_concatenate(slices)

In [18]:
print(final_pca_result.shape)
print(final_pca_result[:2])
print(final_pca_result[0, np.where(final_pca_result[0] == final_pca_result[1])[0]])

(5, 16)
[[ -61.67743206 -182.72352141 -190.36957254  391.67921109  -93.03807272
   -60.96527269  319.69159956  438.04435365  817.19379987  -84.17123842
   -64.99054362  -12.84645103 -300.750603   -263.12937379  701.74759977
  -250.92240277]
 [ 718.69830973  340.45873959   42.02717849  -17.60801766 -437.71127115
   684.33695898 -258.990001    -37.94636295 -195.67801175  -89.07605754
    95.64201766  684.95452296 -284.16487703  776.34687675 -135.89779155
  -215.70089647]]
[]


In [ ]:
# when we do pca on the whole dataset, i.e. not splitting the first dim, then we are back at the initial approach (1000, flattened FM stack per video) and can create up to 800 PCs, right?

In [ ]:
#-----------------------------------------------------------------------

In [ ]:
# zip pca folder
directory_to_zip = "PCA_channel_0.95"  # Replace with your directory name
output_filename = "PCA_channel_0.95_dataset"  # Replace with your desired output name
output_path = os.path.join(os.getcwd(), output_filename)
shutil.make_archive(output_path, 'zip', directory_to_zip)